In [1]:
%matplotlib inline

In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from vtk_rw import read_vtk
import seaborn as sns
from plotting import plot_surf_stat_map
import matplotlib
from graphs import graph_from_mesh, dijkstra, shortest_path
import gdist

/home/raid3/huntenburg/workspace/anaconda2/envs/myelinconnect/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
sub='BP4T'

In [4]:
simple_v, simple_f, _ = read_vtk('/scr/ilz3/myelinconnect/groupavg/indv_space/%s/lowres_rh_d_def.vtk'%sub)

In [5]:
complex_v, complex_f, _ = read_vtk('/scr/ilz3/myelinconnect/struct/surf_rh/orig/mid_surface/%s_rh_mid.vtk'%sub)

In [6]:
mapping_ideal = np.load('/scr/ilz3/myelinconnect/all_data_on_simple_surf/seeds_ideal/%s_rh_highres2lowres_seeds.npy'%sub)

In [7]:
voronoi_seed_idx = mapping_ideal

In [8]:
simple_graph=graph_from_mesh(simple_v, simple_f, edge_length=True)

In [9]:
complex_graph=graph_from_mesh(complex_v, complex_f, edge_length=True)

In [10]:
df = pd.DataFrame(columns=['simple e0', 'simple e1', 'simple len', 
                           'complex e0', 'complex e1', 'complex len'],
                      index=np.arange(len(simple_graph.edges())))

In [11]:
simple_v = simple_v.astype(np.float64)
simple_f = simple_f.astype(np.int32)

In [12]:
complex_v = complex_v.astype(np.float64)
complex_f = complex_f.astype(np.int32)

In [ ]:
count = 0
for e in simple_graph.edges_iter():
    
    dist = gdist.compute_gdist(vertices=complex_v, 
                        triangles=complex_f, 
                        source_indices=np.asarray([voronoi_seed_idx[e[0]]], dtype='int32'), 
                        target_indices=np.asarray([voronoi_seed_idx[e[1]]], dtype='int32')
                       )
    
    df['simple e0'][count] = e[0]
    df['simple e1'][count] = e[1]
    df['simple len'][count] = simple_graph[e[0]][e[1]]

    df['complex e0'][count] = voronoi_seed_idx[e[0]]
    df['complex e1'][count] = voronoi_seed_idx[e[1]]
    df['complex len'][count] = dist
    
    count += 1